# Tensorflow keras integration
 - Base keras model
 - Tensorflow to control the training process
 - Combine Tensorflow layers and keras layers
 - Use tensorboard to show the net & the training process

In [1]:
# Header
# Basic libraries & options
from __future__ import print_function

#Basic libraries
import numpy as np
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)
import time

# Select GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#Show images
import matplotlib.pyplot as plt
%matplotlib inline
# plt configuration
plt.rcParams['figure.figsize'] = (10, 10)        # size of images
plt.rcParams['image.interpolation'] = 'nearest'  # show exact image
plt.rcParams['image.cmap'] = 'gray'  # use grayscale 


Tensorflow version:  1.4.0


## Base keras model

In [2]:
# Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.


In [3]:
# Base Keras Model
images = tf.keras.layers.Input(batch_shape=(None, 28, 28), dtype='float32', name='Images')

flat   = tf.keras.layers.Flatten(name='Flat_image')(images)
dense  = tf.keras.layers.Dense(500, activation='relu', name='Dense_layer')(flat)
output = tf.keras.layers.Dense(10, activation='softmax', name='Dense_output')(dense)

model_linear = tf.keras.models.Model(inputs=images, outputs=output)

# Train
model_linear.compile(loss='sparse_categorical_crossentropy',
                     optimizer=tf.keras.optimizers.SGD(lr=0.01), metrics=['accuracy'])

history_linear = model_linear.fit(X_train, y_train, batch_size=128, epochs=50,
                                  verbose=1, validation_data=(X_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s - loss: 1.0890 - acc: 0.7698 - val_loss: 0.5985 - val_acc: 0.8653
Epoch 2/50
60000/60000 [==============================] - 2s - loss: 0.5208 - acc: 0.8729 - val_loss: 0.4343 - val_acc: 0.8905
Epoch 3/50
60000/60000 [==============================] - 2s - loss: 0.4196 - acc: 0.8898 - val_loss: 0.3739 - val_acc: 0.9029
Epoch 4/50
60000/60000 [==============================] - 2s - loss: 0.3736 - acc: 0.8987 - val_loss: 0.3413 - val_acc: 0.9087
Epoch 5/50
60000/60000 [==============================] - 2s - loss: 0.3453 - acc: 0.9050 - val_loss: 0.3198 - val_acc: 0.9137
Epoch 6/50
60000/60000 [==============================] - 2s - loss: 0.3252 - acc: 0.9097 - val_loss: 0.3031 - val_acc: 0.9167
Epoch 7/50
60000/60000 [==============================] - 2s - loss: 0.3096 - acc: 0.9140 - val_loss: 0.2898 - val_acc: 0.9209
Epoch 8/50
60000/60000 [==============================] - 2s 

# Tensorflow to control the training process
    - use a batch generator
    - Implement a basic training process
        - Create placeholders
        - Create a loss function
        - Create a trainer
        - Define an accuracy metric

In [4]:
# Start interactive session
sess = tf.InteractiveSession()


In [5]:
# Define input placeholders
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

# Define model using Keras layers
flat   = tf.keras.layers.Flatten(name='Flat_image')(x)
dense  = tf.keras.layers.Dense(500, activation='relu', name='Dense_layer')(flat)
output = tf.keras.layers.Dense(10, activation='relu', name='Dense_output')(dense)

# Define the Tensorflow Loss
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)

# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [6]:
# Define a generator to data. Same code to an HDF5 source
def batch_generator(X, y, batch_size=64):
    data_size = X.shape[0]
    # Randomize batches in each epoch
    batch_randomized = np.random.permutation(range(0, data_size-batch_size, batch_size))
    # Iterate over each batch
    for batch in batch_randomized:
        x_batch = X[batch : batch+batch_size]
        y_batch = y[batch : batch+batch_size]
        yield x_batch, y_batch


In [7]:
# Intialize vars
sess.run(tf.global_variables_initializer())


In [8]:
# Basic iterator over the epochs and the batches to execute the trainer. 
batch_size = 128
num_epoch = 50
for epoch in range(num_epoch):
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})
    print(epoch, accuracy.eval(feed_dict={x: X_test, y: y_test}))


0 0.7828
1 0.8836
2 0.8999
3 0.9081
4 0.9129
5 0.916
6 0.9218
7 0.9254
8 0.9269
9 0.9284
10 0.9298
11 0.9323
12 0.9347
13 0.9357
14 0.9379
15 0.9381
16 0.9401
17 0.9414
18 0.9409
19 0.9427
20 0.9442
21 0.9448
22 0.9448
23 0.9471
24 0.9468
25 0.9485
26 0.9479
27 0.9495
28 0.9503
29 0.9511
30 0.9524
31 0.9526
32 0.9539
33 0.9539
34 0.9549
35 0.9553
36 0.9557
37 0.9565
38 0.957
39 0.9576
40 0.9588
41 0.9583
42 0.9588
43 0.9593
44 0.9594
45 0.9603
46 0.9614
47 0.9614
48 0.9617
49 0.9621


In [9]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True

while stop:
    #Train
    epoch += 1
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})

    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    if epoch%10==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

Epoch:  10 Accuracy:  0.731
Epoch:  20 Accuracy:  0.7429
Epoch:  30 Accuracy:  0.7489
Epoch:  40 Accuracy:  0.7532
Epoch:  50 Accuracy:  0.7566
Epoch:  60 Accuracy:  0.7594
Epoch:  70 Accuracy:  0.7622
Epoch:  80 Accuracy:  0.7637
Epoch:  90 Accuracy:  0.7656
Epoch:  100 Accuracy:  0.7666
Epoch:  110 Accuracy:  0.768
Epoch:  120 Accuracy:  0.7689
Epoch:  130 Accuracy:  0.7695
STOP. Accuracy:  0.7692


In [10]:
#When finish, close the interactive session
sess.close()

# Reset the graph to the next experiments
tf.reset_default_graph()

# Tensorflow layers and keras layers
    - Combine Keras layers with tensorflow layers
    - Monitor layers in tensorboard

In [12]:
# Start interactive session
sess = tf.InteractiveSession()


# Convolutional model
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

image = tf.reshape(x,[-1,28,28,1])

conv1 = tf.keras.layers.Conv2D(20, (5,5))(image)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = tf.keras.layers.Conv2D(20, (5,5))(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
flat = tf.keras.layers.Flatten(name='Flat_image')(pool2)
print('Flat layer: ', flat)


# Tensorflow Dense layer
W_dense = tf.Variable(tf.truncated_normal([320, 500], stddev=0.1), name='W_dense')
b_dense = tf.Variable(tf.constant(0.1, shape=[500]), name='b_dense')
dense1 = tf.nn.relu(tf.matmul(flat, W_dense) + b_dense)
#dense1 = layers.Dense(500, activation='relu', name='Dense_1')(flat)


output = tf.keras.layers.Dense(10, activation='linear', name='Dense_output')(dense1)

# Define the Tensorflow Loss
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)

# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Flat layer:  Tensor("Flat_image_2/Reshape:0", shape=(?, 320), dtype=float32)


In [13]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True

while stop:
    #Train
    epoch += 1
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})

    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    if epoch%1==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

Epoch:  1 Accuracy:  0.912
Epoch:  2 Accuracy:  0.9403
Epoch:  3 Accuracy:  0.9502
Epoch:  4 Accuracy:  0.9619
Epoch:  5 Accuracy:  0.9662
Epoch:  6 Accuracy:  0.9688
Epoch:  7 Accuracy:  0.9719
Epoch:  8 Accuracy:  0.9732
Epoch:  9 Accuracy:  0.977
Epoch:  10 Accuracy:  0.98
Epoch:  11 Accuracy:  0.9788
Epoch:  12 Accuracy:  0.9799
Epoch:  13 Accuracy:  0.9777
Epoch:  14 Accuracy:  0.9806
Epoch:  15 Accuracy:  0.9817
Epoch:  16 Accuracy:  0.9823
Epoch:  17 Accuracy:  0.9821
Epoch:  18 Accuracy:  0.9824
Epoch:  19 Accuracy:  0.9826
Epoch:  20 Accuracy:  0.9833
Epoch:  21 Accuracy:  0.9819
Epoch:  22 Accuracy:  0.9788
Epoch:  23 Accuracy:  0.9835
Epoch:  24 Accuracy:  0.9826
Epoch:  25 Accuracy:  0.9829
Epoch:  26 Accuracy:  0.9843
Epoch:  27 Accuracy:  0.9848
Epoch:  28 Accuracy:  0.9841
Epoch:  29 Accuracy:  0.9842
Epoch:  30 Accuracy:  0.9852
Epoch:  31 Accuracy:  0.9854
Epoch:  32 Accuracy:  0.9836
Epoch:  33 Accuracy:  0.9849
Epoch:  34 Accuracy:  0.9853
Epoch:  35 Accuracy:  0.984

In [14]:
#When finish, close the interactive session
sess.close()

# Reset the graph to the next experiments
tf.reset_default_graph()

# Use tensorboard to show the net & the training process.
    - The same previous convolutional model with the commands that need tensorboard

Based on https://www.tensorflow.org/how_tos/summaries_and_tensorboard/index.html 

In [15]:
def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/'   + name, mean)
        tf.summary.scalar('sttdev/' + name, tf.sqrt(tf.reduce_mean(tf.square(var - mean))))
        tf.summary.scalar('max/'    + name, tf.reduce_max(var))
        tf.summary.scalar('min/'    + name, tf.reduce_min(var))
        tf.summary.histogram(name, var)


In [16]:
# Start interactive session
sess = tf.InteractiveSession()


# Convolutional model
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

image = tf.reshape(x,[-1,28,28,1])

with tf.name_scope('Conv1') as scope:
    conv1 = tf.keras.layers.Conv2D(20, (5,5))(image)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    variable_summaries(pool1, "pool1_summary")

with tf.name_scope('Conv2') as scope:
    conv2 = tf.keras.layers.Conv2D(20, (5,5))(pool1)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    variable_summaries(pool2, "pool2_summary")

with tf.name_scope('Dense') as scope:
    flat = tf.keras.layers.Flatten(name='Flat_image')(pool2)
    # Tensorflow Dense layer
    W_dense = tf.Variable(tf.truncated_normal([320, 500], stddev=0.1), name='W_dense')
    variable_summaries(W_dense, "W_dense_summary") # Summaries of the layer weigths
    b_dense = tf.Variable(tf.constant(0.1, shape=[500]), name='b_dense')
    variable_summaries(b_dense, "b_dense_summary") # Summaries of the layer weigths
    dense1 = tf.nn.relu(tf.matmul(flat, W_dense) + b_dense)
    variable_summaries(dense1, "dense1_summary") # Summaries of the layer output

    output = layers.Dense(10, activation='linear', name='Dense_output')(dense1)

# Define the Tensorflow Loss
#with tf.name_scope('loss') as scope:
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)
loss_summary = tf.summary.scalar("loss", cross_entropy)
    
# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
#with tf.name_scope('acc') as scope:
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_summary = tf.summary.scalar("acc", accuracy)

# Add summaries to the graph
summaries_dir = '/tmp/tensorboard/tf'
with tf.name_scope('summaries') as scope:
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
    test_writer  = tf.summary.FileWriter(summaries_dir + '/test')

    

NameError: name 'layers' is not defined

In [ ]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True
while stop:
    #Train
    epoch += 1
    counter = 0
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})
        
        counter += 1
        if counter%10 == 0:
            summary_str = merged.eval(feed_dict={x: x_batch, y: y_batch}) #TENSORBOARD
            train_writer.add_summary(summary_str, epoch) #TENSORBOARD
            
                
    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    
    summary_str = merged.eval(feed_dict={x: X_test, y: y_test}) #TENSORBOARD 
    test_writer.add_summary(summary_str, epoch) #TENSORBOARD 
      
    if epoch%1==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

In [ ]:
sess.close()
tf.reset_default_graph()

At the end execute tensorboar with:

    cd /tmp

    tensorboard --logdir=./tensorboard

And accest to it in:

    http://<My_server_IP>:6006
